In [4]:
import os
import numpy as np
from time import time
from tqdm import trange
from keras.models import Model
from keras.applications import VGG19
from keras.applications.vgg19 import preprocess_input
from keras.preprocessing.image import load_img, img_to_array

In [2]:
def get_img(img_dir):
    """"""
    channel = 3
    target_size = 224, 224
    img_names = os.listdir(img_dir)
    X_img = np.zeros((len(img_names), *target_size, channel), dtype = np.float32)
    damaged_idx = []
    damaged_name = []
    
    # reading in the images and preprocessing
    for i in trange(len(img_names)):
        img_name = img_names[i]
        img_path = os.path.join(img_dir, img_name)
        try:
            img = load_img(img_path, target_size = target_size)
            x = img_to_array(img)
            X_img[i] = x
        except OSError:
            X_img[i] = 0
            damaged_idx.append(i)
            damaged_name.append(img_name)
    
    mask = np.ones(len(img_names), dtype = np.bool)
    mask[damaged_idx] = False
    X_img = X_img[mask]
    X_img = preprocess_input(X_img)
    
    damaged = dict(zip(damaged_idx, damaged_name))
    return X_img, damaged

In [5]:
img_dir = 'image'
X_img, damaged = get_img(img_dir)
X_img.shape

 45%|████▍     | 12137/27225 [00:30<00:37, 400.38it/s]

NameError: name 'idx' is not defined

 45%|████▍     | 12137/27225 [00:50<01:02, 242.69it/s]

In [ ]:
damaged

In [10]:
# preprocess_input from imagenet_utils is for ResNet only, not for Inception and Xception net.
base_model = VGG19(weights = 'imagenet')
model = Model(inputs = base_model.inputs, outputs = base_model.get_layer('fc2').output)
preds = model.predict(X_img, verbose = 1)

100/100 [==============================] - 51s    


In [3]:
img_dir = 'image'

start = time()
preds = get_img_features(img_dir)
end = time() - start
end

16516.147595882416

In [7]:
import joblib

In [6]:
import jupyterthemes
jupyterthemes.__version__

'0.13.9'

In [8]:
save_path = 'img_features.pkl'
joblib.dump(preds, save_path) 

['img_features.pkl']

In [9]:
test = joblib.load(save_path) 

In [22]:
preds

array([[ 2.47853756,  1.36861718,  4.33596611, ...,  6.38603115,
         0.        ,  0.        ],
       [ 0.61399281,  0.        ,  0.64605051, ...,  0.        ,
         0.41054115,  0.        ],
       [ 0.        ,  0.        ,  0.66792887, ...,  0.88802922,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  5.98956013,
         1.94405317,  0.        ],
       [ 0.49130306,  0.84355855,  0.35405356, ...,  2.65636301,
         0.81746984,  0.41911855],
       [ 0.        ,  0.        ,  0.7070117 , ...,  3.54142141,
         0.79095614,  0.        ]], dtype=float32)

In [19]:
from joblib import Memory

In [20]:
?Memory

In [21]:
?np.load

In [ ]:
import joblib
import os

cachedir = 'cache'
if not os.path.isdir(cachedir): os.mkdir(cachedir)
mem = joblib.Memory(cachedir=cachedir, verbose=True)

@mem.cache
def my_long_function(i):
	return i + i

# first call is done normally
print my_long_function(1)
# second call is loaded from cache directory
print my_long_function(1)